In [1]:
import pickle
import re
import pandas as pd

import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize

In [2]:
filename = 'model.pickle'

obj = pickle.load(open(filename, 'rb'))

In [3]:
model = obj['model']
stopwords = obj['stopwords']
lemmatizer = obj ['lemmatizer']
wordnet_map = obj ['wordnet_map']
vectorizer = obj ['vectorizer']

In [4]:
#prediction pipeline with text vectorizer and model
def prediction(text_input, STOPWORDS, lemmatizer, wordnet_map, vectorizer, Multinomial_NB):
    
    # Transform words to lower case
    text=text_input.lower()
    # Remove punctuation
    text=re.sub('[^a-zA-Z]', ' ', text)
    # Remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    # Remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    # Remove stop words like : and is a and the
    text=" ".join([word for word in text.split() if word not in STOPWORDS])
    # Find base word for all words in the sentence
    pos_tagged_text=nltk.pos_tag(text.split())
    text=" ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])
    
    # specific
    text = re.sub(r"won\'t", "will not", text); text = re.sub(r"can\'t", "can not", text)
    # general
    text = re.sub(r"n\'t", " not", text); text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text); text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text); text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text); text = re.sub(r"\'m", " am", text)
    print(text)
    text = np.array(text).reshape(1,)
    vec_text = vectorizer.transform(text)
    output = Multinomial_NB.predict(vec_text)
    
    #creating dataframe
    cols = ['text','emotion_0','emotion_1','emotion_2','emotion_3','emotion_4',
            'emotion_5','emotion_6','emotion_7','emotion_8']
    vals = [[text_input] + list(output[0])]
    df = pd.DataFrame(vals, columns=cols)
    
    return df

In [5]:
#Source[BBC]: https://www.bbc.com/news/world-asia-56546920
text_input= "Myanmar coup: Dozens killed as army opens fire on protesters during deadliest day.Security forces were out in strength trying to prevent rallies.Local news site Myanmar Now put the death toll at 114, while the United Nations said it was receiving reports of scores killed and hundreds more injured."

#Prediction output
output= prediction(text_input, stopwords, lemmatizer, wordnet_map, vectorizer, model)
output

myanmar coup dozen kill army open fire protester deadliest day security force strength try prevent rally local news site myanmar put death toll united nation say receive report score kill hundred injure


,text,emotion_0,emotion_1,emotion_2,emotion_3,emotion_4,emotion_5,emotion_6,emotion_7,emotion_8
0,Myanmar coup: Dozens killed as army opens fire...,0,1,0,0,0,1,0,1,0


In [6]:
#prediction pipeline with text vectorizer and model
def prediction(text_input, STOPWORDS, lemmatizer, wordnet_map, vectorizer, Multinomial_NB):
    
    # Transform words to lower case
    text=text_input.lower()
    # Remove punctuation
    text=re.sub('[^a-zA-Z]', ' ', text)
    # Remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    # Remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    # Remove stop words like : and is a and the
    text=" ".join([word for word in text.split() if word not in STOPWORDS])
    # Find base word for all words in the sentence
    pos_tagged_text=nltk.pos_tag(text.split())
    text=" ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])
    
    # specific
    text = re.sub(r"won\'t", "will not", text); text = re.sub(r"can\'t", "can not", text)
    # general
    text = re.sub(r"n\'t", " not", text); text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text); text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text); text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text); text = re.sub(r"\'m", " am", text)
#     print(text)
    text = np.array(text).reshape(1,)
    vec_text = vectorizer.transform(text)
    output = Multinomial_NB.predict(vec_text)
    
    cols = ['emotion_0','emotion_1','emotion_2','emotion_3','emotion_4',
            'emotion_5','emotion_6','emotion_7','emotion_8']
    output_values = [False if x == 0 else True for x in output[0]]  # np values are replaced with boolean
 
    return { key: value for key, value in zip(cols, output_values)}


In [8]:
#Source[BBC]: https://www.bbc.com/news/world-asia-56546920
text_input= "Myanmar coup: Dozens killed as army opens fire on protesters during deadliest day.Security forces were out in strength trying to prevent rallies.Local news site Myanmar Now put the death toll at 114, while the United Nations said it was receiving reports of scores killed and hundreds more injured."

#Prediction output
output= prediction(text_input, stopwords, lemmatizer, wordnet_map, vectorizer, model)
output

{'emotion_0': False,
 'emotion_1': True,
 'emotion_2': False,
 'emotion_3': False,
 'emotion_4': False,
 'emotion_5': True,
 'emotion_6': False,
 'emotion_7': True,
 'emotion_8': False}

In [67]:
!ls *.p*

model.pickle


In [9]:
!aws s3 cp ./model.pickle s3://com.msarica.ds/model.pickle

upload: ./model.pickle to s3://com.msarica.ds/model.pickle        


In [14]:
!aws s3 cp ./Part3.py s3://com.msarica.ds/server.txt

upload: ./Part3.py to s3://com.msarica.ds/server.txt           


In [12]:
# aws s3 cp s3://com.msarica.ds/server.txt ./server.txt